In [ ]:
pip install dgllife

In [ ]:
pip install RDKit

In [16]:
import pandas as pd
import dgl
import dgllife

In [3]:
db = pd.read_csv('../data/HIV.csv')

In [4]:
db

,smiles,activity,HIV_active
0,CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)...,CI,0
1,C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3...,CI,0
2,CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21,CI,0
3,Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1,CI,0
4,O=S(=O)(O)CCS(=O)(=O)O,CI,0
...,...,...,...
41122,CCC1CCC2c3c([nH]c4ccc(C)cc34)C3C(=O)N(N(C)C)C(...,CI,0
41123,Cc1ccc2[nH]c3c(c2c1)C1CCC(C(C)(C)C)CC1C1C(=O)N...,CI,0
41124,Cc1ccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)C...,CI,0
41125,Cc1cccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)...,CI,0


In [84]:
import random

In [80]:
CI_liste = db.index[db['activity'] == 'CI'].tolist()

In [89]:
CM_CA_liste = db.index[db['activity'] != 'CI'].tolist()

In [85]:
CI_kept = random.sample(CI_liste, 1000)

In [104]:
residual = sorted(CI_kept + CM_CA_liste)

In [106]:
residual_db = db.filter(items = residual, axis=0)

In [107]:
residual_db.value_counts('activity')

activity
CM    1039
CI    1000
CA     404
Name: count, dtype: int64

In [108]:
residual_db

,smiles,activity,HIV_active
11,O=C(O)Cc1ccc(SSc2ccc(CC(=O)O)cc2)cc1,CM,1
16,NNP(=S)(NN)c1ccccc1,CM,1
29,c1ccc2nsnc2c1,CI,0
80,O=Nc1ccc(O)c(N=O)c1O,CM,1
107,CCN(CC)CCCCCCNc1cc(OC)cc2c(C)ccnc12,CI,0
...,...,...,...
41092,Cc1cn(C2CC3C(COC(CCC[Se]c4ccccc4)N3O)O2)c(=O)[...,CM,1
41093,Cc1cn(C2CC3C(COC(CCCC[Se]c4ccccc4)N3O)O2)c(=O)...,CM,1
41098,Cc1cn(C2CC3C(COC(CC[Se]C#N)N3O)O2)c(=O)[nH]c1=O,CM,1
41099,C[Se]CCC1OCC2OC(n3cc(C)c(=O)[nH]c3=O)CC2N1O,CA,1


In [109]:
molecules_smiles = residual_db['smiles'].to_list()

In [110]:
activities = residual_db['activity'].to_list()

In [111]:
from rdkit import Chem
import torch
def featurize_atoms(mol):
    feats = []
    for atom in mol.GetAtoms():
        feats.append(atom.GetAtomicNum())
    return {'atomic': torch.tensor(feats).reshape(-1, 1).float()}


def featurize_bonds(mol):
    feats = []
    bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE,
                 Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]
    for bond in mol.GetBonds():
        btype = bond_types.index(bond.GetBondType())
        # One bond between atom u and v corresponds to two edges (u, v) and (v, u)
        feats.extend([btype, btype])
    return {'type': torch.tensor(feats).reshape(-1, 1).float()}

In [112]:
from tqdm import tqdm

def convert_graphs(smile_graphs):
    dgl_graphs=[]
    for g in tqdm(smile_graphs):
        dgl_graphs.append(dgllife.utils.smiles_to_bigraph(g,node_featurizer=featurize_atoms,
                     edge_featurizer=featurize_bonds))
    return dgl_graphs

In [113]:
molecules_dgl = convert_graphs(molecules_smiles)

100%|██████████| 2443/2443 [00:10<00:00, 222.44it/s]


In [114]:
molecules_smiles[0]

'O=C(O)Cc1ccc(SSc2ccc(CC(=O)O)cc2)cc1'

In [115]:
molecules_dgl[0].ndata

{'atomic': tensor([[ 8.],
        [ 8.],
        [ 8.],
        [ 6.],
        [ 6.],
        [ 6.],
        [16.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 8.],
        [ 6.],
        [ 6.],
        [ 6.],
        [ 6.],
        [16.],
        [ 6.]])}

In [116]:
activities_index = [ 0 if act=='CI' else ( 1 if act=='CM' else 2) for act in activities]

In [117]:
 graph_labels = {"glabel": torch.tensor(activities_index)}

ne pas réexécuter : utilisé pour sauvegarder la donnée

In [118]:
#dgl.save_graphs('../data/HIV_dgl_graphs', molecules_dgl, labels=graph_labels)

code à utiliser pour loader le graph et les labels, NB : 0 = inactif, 1 = moyen actif, 2 = actif

In [75]:
glist, label_dict = dgl.load_graphs("../data/HIV_dgl_graphs")